In [ ]:
#!pip install onnxruntime

In [4]:
!pip install onnxruntime-gpu

In [5]:
import onnxruntime
onnxruntime.get_device()

'GPU'

In [6]:
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from collections import OrderedDict
import threading



In [7]:
def video_to_frames(video_path):
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    frames = []

    while success:
        frames.append(image)
        success, image = vidcap.read()
        count += 1

    return frames

In [10]:
import os
import math
import cv2
import numpy as np
# import onnxruntime
# from onnxruntime.capi import _pybind_state as C

__labels = [
'gun','Rifle'
]
def _read_image(img, target_size=320):

  img_height, img_width = img.shape[:2]
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  aspect = img_width / img_height

  if img_height > img_width:
      new_height = target_size
      new_width = int(round(target_size * aspect))
  else:
      new_width = target_size
      new_height = int(round(target_size / aspect))

  resize_factor = math.sqrt(
      (img_width**2 + img_height**2) / (new_width**2 + new_height**2)
  )

  img = cv2.resize(img, (new_width, new_height))

  pad_x = target_size - new_width
  pad_y = target_size - new_height

  pad_top, pad_bottom = [int(i) for i in np.floor([pad_y, pad_y]) / 2]
  pad_left, pad_right = [int(i) for i in np.floor([pad_x, pad_x]) / 2]

  img = cv2.copyMakeBorder(
      img,
      pad_top,
      pad_bottom,
      pad_left,
      pad_right,
      cv2.BORDER_CONSTANT,
      value=[0, 0, 0],
  )

  img = cv2.resize(img, (target_size, target_size))

  image_data = img.astype("float32") / 255.0  # normalize
  image_data = np.transpose(image_data, (2, 0, 1))
  image_data = np.expand_dims(image_data, axis=0)

  return image_data, resize_factor, pad_left, pad_top

def _postprocess(output, resize_factor, pad_left, pad_top):
  outputs = np.transpose(np.squeeze(output[0]))
  rows = outputs.shape[0]
  boxes = []
  scores = []
  class_ids = []

  for i in range(rows):
      classes_scores = outputs[i][4:]
      max_score = np.amax(classes_scores)

      if max_score >= 0.2:
          class_id = np.argmax(classes_scores)
          x, y, w, h = outputs[i][0], outputs[i][1], outputs[i][2], outputs[i][3]
          left = int(round((x - w * 0.5 - pad_left) * resize_factor))
          top = int(round((y - h * 0.5 - pad_top) * resize_factor))
          width = int(round(w * resize_factor))
          height = int(round(h * resize_factor))
          class_ids.append(class_id)
          scores.append(max_score)
          boxes.append([left, top, width, height])

  indices = cv2.dnn.NMSBoxes(boxes, scores, 0.25, 0.45)

  detections = []
  for i in indices:
      box = boxes[i]
      score = scores[i]
      class_id = class_ids[i]
      detections.append(
          {"class": __labels[class_id], "score": float(score), "box": box}
      )

  return detections

def frames_to_video(detected_frames, output_video_path):
  height, width, _ = detected_frames[0].shape
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (width, height))

  for frame in detected_frames:
      out.write(frame)

  out.release()

global_boxes_lock = threading.Lock()
global_boxes_dict = {}

class Detector:

    def __init__(self,model, providers=None):
        self.onnx_session = onnxruntime.InferenceSession(
           model,
          providers=['CUDAExecutionProvider'],
        )
        model_inputs = self.onnx_session.get_inputs()
        input_shape = model_inputs[0].shape
        self.input_width = input_shape[2]  # 320
        self.input_height = input_shape[3]  # 320
        self.input_name = model_inputs[0].name

    def detect(self, image_path):
      preprocessed_image, resize_factor, pad_left, pad_top = _read_image(
          image_path, self.input_width
      )
      outputs = self.onnx_session.run(None, {self.input_name: preprocessed_image})
      detections = _postprocess(outputs, resize_factor, pad_left, pad_top)

      return detections

    def censor(self, image_path, classes=[], output_path=None, frame_index=None):
        frames = video_to_frames(image_path)
        final = []
        local_boxes = []  # Local list to store boxes for this detector

        for frame_number, frame in enumerate(frames):
            detections = self.detect(frame)
            boxes_for_frame = []

            for detection in detections:
                box = detection.get("box", None)
                if box:
                    x, y, w, h = map(int, box)
                    if 0 <= x < frame.shape[1] and 0 <= y < frame.shape[0] and w > 0 and h > 0:
                        # frame[y:y + h, x:x + w] = cv2.blur(frame[y:y + h, x:x + w], (50, 50))

                        # Appending boxes to the local list for this frame
                        local_boxes.append((frame_number, x, y, w, h))
                        boxes_for_frame.append((x, y, w, h))

            # Acquiring the lock before updating the global dictionary
            global_boxes_lock.acquire()
            if frame_number not in global_boxes_dict:
                global_boxes_dict[frame_number] = []
            global_boxes_dict[frame_number].extend(boxes_for_frame)
            global_boxes_lock.release()

            final.append(frame)
        frames = blur_detected_boxes(frames)

        return frames

def blur_detected_boxes(frames):
    for frame_index, boxes in global_boxes_dict.items():
        if boxes:  # Checking if there are detected boxes for this frame
            image_boxes = boxes
            for box in image_boxes:
                x, y, w, h = box
                frames[frame_index][y:y + h, x:x + w] = cv2.blur(frames[frame_index][y:y + h, x:x + w], (50, 50))
    return frames



def initialize_detector(model):
    return Detector(model)


if __name__ == "__main__":
    models=["/kaggle/input/cv-project/guns.onnx",'/kaggle/input/cv-project/rifle.onnx']

# Number of concurrent workers 
    num_workers = len(models)

# Creating a ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
      detectors = list(executor.map(initialize_detector, models))
          # Process frames in parallel using each detector
      concurrent_results = []
      for detector in detectors:
        results=detector.censor('/kaggle/input/cv-project/output_video_cropped.mp4')  # Performing detection on frames using each detector
        # concurrent_results.append(results)










In [11]:

output_video_path = 'output.mp4'
frames_to_video(results,output_video_path)

In [2]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def crop_video(input_file, output_file, start_time, end_time):
    ffmpeg_extract_subclip(input_file, start_time, end_time, targetname=output_file)

# Example usage
input_file = '/content/input.mp4'  
output_file = 'output_video_cropped.mp4'  
start_time = 240+35  # Start time in seconds
end_time = 300+23  # End time in seconds

crop_video(input_file, output_file, start_time, end_time)


Moviepy - Running:

>>> "+ " ".join(cmd)

Moviepy - Command successful


In [1]:
# installing the video from youtube
!pip install pytube

from pytube import YouTube

def Download(link):
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    try:
        youtubeObject.download()
    except:
        print("An error has occurred")
    print("Download is completed successfully")
Download('https://www.youtube.com/watch?v=dXbOU2_Yh7A&t=278s')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.0 MB/s eta 0:00:00a 0:00:01



Download is completed successfully
